In [1]:
import sqlite3

In [2]:
db = sqlite3.connect('test_db.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates") # Convenient in case you want to start over
cursor.execute("DROP TABLE IF EXISTS contributors") # Convenient in case you want to start over

In [3]:
# This strange command turns out FOREIGN KEY support in SQLite.
# By default, this support is off.
# We'll talk about foreign keys later on.  Just setting things up here.
cursor.execute("PRAGMA foreign_keys=1")

In [4]:
cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

db.commit() # Commit changes to the database

In [5]:
f = open('candidates.txt','r')
data = f.read()
f.close()

In [6]:
d = data.split('\n')
d

['id|first_name|last_name|middle_name|party',
 '33|Joseph|Biden||D',
 '36|Samuel|Brownback||R',
 '34|Hillary|Clinton|R.|D',
 '39|Christopher|Dodd|J.|D',
 '26|John|Edwards||D',
 '22|Rudolph|Giuliani||R',
 '24|Mike|Gravel||D',
 '16|Mike|Huckabee||R',
 '30|Duncan|Hunter||R',
 '31|Dennis|Kucinich||D',
 '37|John|McCain||R',
 '20|Barack|Obama||D',
 '32|Ron|Paul||R',
 '29|Bill|Richardson||D',
 '35|Mitt|Romney||R',
 '38|Tom|Tancredo||R',
 '41|Fred|Thompson|D.|R',
 '']

In [7]:
for i in range(1, len(d)-1):
    data = d[i].split('|')
    cursor.execute('''INSERT INTO candidates
               (id, first_name, last_name, middle_init, party)
               VALUES (?, ?, ?, ?, ?)''', 
                (data[0], data[1], data[2], data[3], data[4]))
db.commit()

In [8]:
cursor.execute("SELECT * FROM candidates")
all_rows = cursor.fetchall()
print(all_rows)

[(16, 'Mike', 'Huckabee', '', 'R'), (20, 'Barack', 'Obama', '', 'D'), (22, 'Rudolph', 'Giuliani', '', 'R'), (24, 'Mike', 'Gravel', '', 'D'), (26, 'John', 'Edwards', '', 'D'), (29, 'Bill', 'Richardson', '', 'D'), (30, 'Duncan', 'Hunter', '', 'R'), (31, 'Dennis', 'Kucinich', '', 'D'), (32, 'Ron', 'Paul', '', 'R'), (33, 'Joseph', 'Biden', '', 'D'), (34, 'Hillary', 'Clinton', 'R.', 'D'), (35, 'Mitt', 'Romney', '', 'R'), (36, 'Samuel', 'Brownback', '', 'R'), (37, 'John', 'McCain', '', 'R'), (38, 'Tom', 'Tancredo', '', 'R'), (39, 'Christopher', 'Dodd', 'J.', 'D'), (41, 'Fred', 'Thompson', 'D.', 'R')]


In [9]:
cursor.execute("SELECT * FROM candidates WHERE first_name = 'Joseph'")
all_rows = cursor.fetchall()
print(all_rows)

[(33, 'Joseph', 'Biden', '', 'D')]


In [10]:
cursor.execute("SELECT first_name FROM candidates")
all_rows = cursor.fetchall()
print(all_rows)

[('Mike',), ('Barack',), ('Rudolph',), ('Mike',), ('John',), ('Bill',), ('Duncan',), ('Dennis',), ('Ron',), ('Joseph',), ('Hillary',), ('Mitt',), ('Samuel',), ('John',), ('Tom',), ('Christopher',), ('Fred',)]


In [11]:
cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount INTEGER, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')

db.commit()

In [12]:
contributors = [("Agee", "Steven", "", "549 Laurel Branch Road", "", "Floyd", "VA", int(24091), int(500), 2007-6-30, 16),
                ("Aaron", "Carole", "", "PO Box 1806", "", "Ogunqui", "ME", int(3907), int(70), 2008-2-7, 34)]

cursor.executemany('INSERT INTO contributors (last_name, first_name, middle_name, street_1, street_2, city, state, zip, amount, date, candidate_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', contributors)
db.commit()

In [13]:
cursor.execute("SELECT last_name FROM contributors where amount < 100")
for c in cursor.fetchall():
    print(c)

('Aaron',)


In [14]:
cursor.execute('INSERT INTO contributors (last_name, first_name, middle_name, street_1, street_2, city, state, zip, amount, date, candidate_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', ("Buckler", "Steve", "", "24351 Armada Dr.", "", "Dana Point", "CA", int(926291), int(50), 2007-7-30, 20))